# Lab 6 - Threat Hunting with Data Science
**Security Data Science**

Sergio Marchena - 16387

UVG - 2022

In [40]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import random
from flare.tools.umbrella import Umbrella
pd.set_option("display.max_columns", None)

## Parte 1 - Filtrado y preprocesamiento 

In [2]:
data = [json.loads(line) for line in open('large_eve.json', 'r')]

### 1. Cargue  la  información  del  archivo  large_even.json  en  una  lista,  muestre  la  cantidad  de registros total (deben ser 746, 909).

In [3]:
print('Hay %d registros.' %len(data))

Hay 746909 registros.


### 2. Debido a que estamos buscando dominios DGA, del total de registros, solamente estamos  interesados  en  los  registros  DNS.  Cargue  únicamente  aquellos  registros  en  cuya  llave  se encuentra “DNS”. 

In [4]:
dns = [d for d in data if d['event_type'] == 'dns']

• Muestre la nueva cantidad de registros filtrados. 

In [5]:
print('Hay %d registros DNS.'%len(dns))

Hay 15749 registros DNS.


• Muestre la información de 2 registros cualesquiera. 

In [6]:
random.sample(dns, 2)

[{'timestamp': '2017-07-22T19:29:18.429632-0500',
  'flow_id': 1239280867249525,
  'pcap_cnt': 3353490,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.205.188',
  'dest_port': 36481,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 21747,
   'rcode': 'NXDOMAIN',
   'rrname': '<root>',
   'rrtype': 'SOA',
   'ttl': 20864}},
 {'timestamp': '2017-07-22T18:29:17.398076-0500',
  'flow_id': 789760764088884,
  'pcap_cnt': 504633,
  'event_type': 'dns',
  'vlan': 150,
  'src_ip': '192.168.207.4',
  'src_port': 53,
  'dest_ip': '192.168.205.188',
  'dest_port': 45036,
  'proto': 'UDP',
  'dns': {'type': 'answer',
   'id': 21925,
   'rcode': 'NXDOMAIN',
   'rrname': 'mirror.its.uidaho.edu'}}]

• Debido a que la data consiste en json anidados, utilice la característica json_normalize para 
normalizar la información y asignarla en un dataframe. 

In [7]:
df = pd.json_normalize(dns)
df

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15744,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,en-us.start3.mozilla.com,NaN,NaN,NXDOMAIN,NaN,NaN
15745,2017-07-22T19:47:03.373435-0500,891876064866556,4441474,dns,120,192.168.207.4,53,192.168.202.172,60654,UDP,answer,33107,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
15746,2017-07-22T19:47:38.146503-0500,2199315651640391,4452917,dns,130,192.168.203.67,50975,192.168.207.4,53,UDP,query,35892,whitecell.localdomain,AAAA,0.0,NaN,NaN,NaN
15747,2017-07-22T19:47:38.146953-0500,2199315651640391,4452918,dns,130,192.168.207.4,53,192.168.203.67,50975,UDP,answer,35892,whitecell.localdomain,NaN,NaN,NXDOMAIN,NaN,NaN


• Como  estamos  buscando  dominios  DGA,  debemos  filtrar  los  registros  DNS  para  aquellos 
registros tipo A (son aquellos que mantienen una dirección IP asociada a un dominio), se debió 
bajar la cantidad a 2849 registros. 

In [8]:
dnsA = df[df['dns.rrtype'] == 'A']
print('Hay %d registros DNS.'%len(dnsA))

Hay 2849 registros DNS.


• Filtre los dominios únicos. 

In [9]:
unique_domains = dnsA.drop_duplicates(subset=['dns.rrname'])
unique_domains

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15632,2017-07-22T19:44:29.056834-0500,787856241843714,4405786,dns,130,192.168.203.69,60088,192.168.207.4,53,UDP,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN
15661,2017-07-22T19:45:10.437871-0500,1612464195546735,4417023,dns,160,192.168.207.4,11162,192.168.206.44,53,UDP,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN
15701,2017-07-22T19:47:38.301392-0500,515168780589392,4452953,dns,120,192.168.202.172,51325,192.168.207.4,53,UDP,query,54310,ocsp.verisign.com,A,0.0,NaN,NaN,NaN
15716,2017-07-22T19:39:02.426497-0500,572502265201153,4319593,dns,180,192.168.198.58,42157,192.168.207.4,53,UDP,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN


In [10]:
domains = unique_domains['dns.rrname'].value_counts()
domains

api.wunderground.com                      1
www.cakephp.org                           1
mirror.sanctuaryhost.com                  1
mirror.team-cymru.org                     1
mirror.umoss.org                          1
                                         ..
sourceforge.net                           1
www.freepbx.org                           1
secure.informaction.com.hackerlabs.vpn    1
www.bigflickrfeed.com                     1
client-software.real.com                  1
Name: dns.rrname, Length: 177, dtype: int64

• Del dataframe de dominios únicos de tipo A, obtenga el TLD (top level domain) utilizando Flare 
como una columna nueva. 

In [11]:
from flare.data_science.features import domain_tld_extract

In [12]:
# Domain TLD Extract
unique_domains['TLD_name'] = unique_domains['dns.rrname'].apply(domain_tld_extract)
unique_domains.sample(10)

/var/folders/3q/f_c12djx2yxc8v2vq4255gcm0000gn/T/ipykernel_99886/629738443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_domains['TLD_name'] = unique_domains['dns.rrname'].apply(domain_tld_extract)


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,TLD_name
2165,2017-07-22T18:29:17.167193-0500,1219113612315929,504139,dns,150,192.168.205.188,38743,192.168.207.4,53,UDP,query,569,mirror.stanford.edu,A,0.0,NaN,NaN,NaN,stanford.edu
8288,2017-07-22T19:01:22.710611-0500,1614005916587987,2292723,dns,150,192.168.205.58,36210,192.168.207.4,53,UDP,query,50616,secure.informaction.com.office.aol.com,A,0.0,NaN,NaN,NaN,aol.com
1398,2017-07-22T18:00:19.859650-0500,1929952060710402,272947,dns,130,192.168.203.62,56942,192.168.207.4,53,UDP,query,21968,download.windowsupdate.com,A,0.0,NaN,NaN,NaN,windowsupdate.com
15661,2017-07-22T19:45:10.437871-0500,1612464195546735,4417023,dns,160,192.168.207.4,11162,192.168.206.44,53,UDP,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN,microsoft.com
2617,2017-07-22T18:29:18.224042-0500,663175193062186,506134,dns,150,192.168.205.188,39667,192.168.207.4,53,UDP,query,9983,centos.mirrors.tds.net,A,0.0,NaN,NaN,NaN,tds.net
2089,2017-07-22T18:24:52.733891-0500,957004625752771,482149,dns,140,192.168.204.60,62393,192.168.207.4,53,UDP,query,1894,google.com,A,0.0,NaN,NaN,NaN,google.com
2159,2017-07-22T18:29:17.165299-0500,94553735267763,504120,dns,150,192.168.205.188,37622,192.168.207.4,53,UDP,query,40896,mirrors.bluehost.com,A,0.0,NaN,NaN,NaN,bluehost.com
1415,2017-07-22T17:58:07.788238-0500,78692298458894,262748,dns,110,192.168.201.57,59523,192.168.207.4,53,UDP,query,41900,api.flickr.com,A,0.0,NaN,NaN,NaN,flickr.com
852,2017-07-22T17:48:55.439985-0500,658504257353393,208390,dns,120,192.168.202.66,36359,192.168.207.4,53,UDP,query,45380,secure.informaction.com.hsd1.pa.comcast.net,A,0.0,NaN,NaN,NaN,comcast.net
1701,2017-07-22T18:16:31.587325-0500,1698247228782141,433198,dns,120,192.168.202.66,37824,192.168.207.4,53,UDP,query,52122,www.sql-ledger.org.hsd1.pa.comcast.net,A,0.0,NaN,NaN,NaN,comcast.net


## Parte 2 – Data Science 

• Utilice su clasificador entrenado en el laboratorio 1, y clasifique los dominios del dataframe de la parte 1. Filtre aquellos claisificados como DGA y muéstrelos. 

In [18]:
import joblib
import whois

from urllib.parse import urlparse

def getDomain(url):
    return urlparse(url).netloc
    
def getProtocol(url):
    return 1 if urlparse(url).scheme == 'https' else 0

def creation_date(domain):
    return whois.whois(domain).creation_date

def getSpecialCharacters(url):
    count_characters = 0
    count_characters += url.count('.')
    count_characters += url.count('-')
    count_characters += url.count('@')
    count_characters += url.count('?')
    count_characters += url.count('&')
    count_characters += url.count('|')
    count_characters += url.count('=')
    count_characters += url.count('_')
    count_characters += url.count('~')
    count_characters += url.count('%')
    count_characters += url.count('/')
    count_characters += url.count('*')
    count_characters += url.count(':')
    count_characters += url.count(',')
    count_characters += url.count(';')
    count_characters += url.count('$')
    count_characters += url.count('%20')
    count_characters += url.count(' ')
    return count_characters

def digitsRatio(x):
  digits = sum(c.isdigit() for c in x)
  length = len(x)
  try:
    proporcion = digits/length
  except:
      proporcion = 0
  return proporcion

In [22]:
new_df = pd.DataFrame()
new_df['longitud_hostname'] = unique_domains['dns.rrname'].apply(getDomain).str.len()
new_df['special_characters'] = unique_domains['dns.rrname'].apply(getSpecialCharacters)
new_df['longitud_url'] = unique_domains['dns.rrname'].str.len()
new_df['ratio_digits_url'] = unique_domains['dns.rrname'].str.count('[0-9]') / unique_domains['dns.rrname'].str.len()
new_df['ratio_digits_domain'] = unique_domains['dns.rrname'].apply(getDomain).str.count('[0-9]') / unique_domains['dns.rrname'].apply(getDomain).str.len()

In [24]:
new_df['longitud_hostname'] = new_df['longitud_hostname'].fillna(0)
new_df['special_characters'] = new_df['special_characters'].fillna(0)
new_df['ratio_digits_domain'] = new_df['ratio_digits_domain'].fillna(0)
new_df['longitud_url'] = new_df['longitud_url'].fillna(0)
new_df['ratio_digits_url'] = new_df['ratio_digits_url'].fillna(0)

In [26]:
import sklearn
from sklearn import metrics, model_selection, tree

modelo = joblib.load('modelo.h5')

In [33]:
predicciones = modelo.predict(new_df)

resultado = []

for i in predicciones:
    if i == 1:
        resultado.append(True)
    else:
        resultado.append(False)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [37]:
unique_domains[resultado]['TLD_name']

163      192.168.22.110phpmyadmin
1195             "192.168.206.56"
1836              192.168.26-27.0
6898              192.168.21.1201
9979              1922.168.22.254
10048        1922.168.22.254.home
10607             192.168.21-28.0
14730        192.168.21-28.0.home
15507             192.168.22.201:
Name: TLD_name, dtype: object

• Utilice  el  clasificador  de  DGA  que  incluye  Flare,  y  clasifique  los  dominios  (agregue  a  su dataframe la columna de clase). Filtre aquellos considerados como DGA y muéstrelos. ¿Son los mismos dominios que los de su modelo? 

In [ ]:
from flare.data_science.features import dga_classifier
from flare.tools.alexa import Alexa

dga_c = dga_classifier()

## Parte 3 – Dominio experto 

• Ahora ya tenemos un listado de dominios reducido y considerado como sospechoso, por lo que debemos aplicar dominio experto para encontrar los verdaderos registros malicioso. Utilizando la lista de Cisco Umbrella incluido en Flare, indique si los dominios encontrados por el clasificador DGA de Flare se encuentran en este top 1000000. 

In [44]:
umbrella = Umbrella(limit = 1000000)

[+] Updated Umbrella Top 1 Million list...


In [75]:
unique_domains['classification'] = resultado

/var/folders/3q/f_c12djx2yxc8v2vq4255gcm0000gn/T/ipykernel_99886/187176299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_domains['classification'] = resultado


In [76]:
values = []

for x in unique_domains['classification']:
    if x:
        values.append(1)
    else:
        values.append(0)

unique_domains['classification'] = values

/var/folders/3q/f_c12djx2yxc8v2vq4255gcm0000gn/T/ipykernel_99886/3274149713.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_domains['classification'] = values


In [78]:
unique_domains['classification'].value_counts()

0    168
1      9
Name: classification, dtype: int64

In [80]:
for i in unique_domains[unique_domains['classification'] == 1]['TLD_name']:
    if umbrella.domain_tld_in_umbrella(i):
        print(i, ' se encuentra en Cisco Umbrella')
    else:
        print(i, ' no se encuentra en Cisco Umbrella')

192.168.22.110phpmyadmin  no se encuentra en Cisco Umbrella
"192.168.206.56"  no se encuentra en Cisco Umbrella
192.168.26-27.0  no se encuentra en Cisco Umbrella
192.168.21.1201  no se encuentra en Cisco Umbrella
1922.168.22.254  no se encuentra en Cisco Umbrella
1922.168.22.254.home  no se encuentra en Cisco Umbrella
192.168.21-28.0  no se encuentra en Cisco Umbrella
192.168.21-28.0.home  no se encuentra en Cisco Umbrella
192.168.22.201:  no se encuentra en Cisco Umbrella


• Si son considerados por el clasificador como DGA, y no están en el top de Cisco Umbrella es muy probable que sean maliciosos. Para confirmar esto podemos buscar la fecha de creación 
del dominio. Cree una función qué en base al dominio, devuelva la fecha de creación de este (utilice la librería whois para esto). 

In [84]:
def funcion1():
    for i in unique_domains[unique_domains['classification'] == 1 ]['TLD_name']:
        try:
            print(i, 'FECHA de CREACION: ', creation_date(i))
        except:
            print(i, 'NA')

• Muestre la fecha de creación para cada uno de los dominios clasificados como DGA. ¿Cuáles son los dominios que podemos confirmar como sospechosos? 

In [85]:
funcion1()

192.168.22.110phpmyadmin FECHA de CREACION:  None
"192.168.206.56" FECHA de CREACION:  None
192.168.26-27.0 FECHA de CREACION:  None
192.168.21.1201 FECHA de CREACION:  None
1922.168.22.254 FECHA de CREACION:  None
1922.168.22.254.home FECHA de CREACION:  None
192.168.21-28.0 FECHA de CREACION:  None
192.168.21-28.0.home FECHA de CREACION:  None
192.168.22.201: FECHA de CREACION:  None


Estos son los dominios que se pueden confirmar como sospechosos, debido a que no estan en Cisco Umbrella y no tienen fecha de creacion. 